# Data preprocessing

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

/Users/mmendoza/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'seq2seq/spa.txt'

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 100000
Number of unique input tokens: 85
Number of unique output tokens: 104
Max sequence length for inputs: 43
Max sequence length for outputs: 77


In [5]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters)])

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Model architecture

In [8]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [9]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model fitting

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 80000 samples, validate on 20000 samples
Epoch 1/10
 7104/80000 [=>............................] - ETA: 16:58 - loss: 1.0381

# Inference

In [11]:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [12]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [14]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [16]:
for seq_index in range(1000):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: ¡Vení!

-
Input sentence: Go.
Decoded sentence: ¡Vení!

-
Input sentence: Go.
Decoded sentence: ¡Vení!

-
Input sentence: Go.
Decoded sentence: ¡Vení!

-
Input sentence: Hi.
Decoded sentence: ¡Aplate!

-
Input sentence: Run!
Decoded sentence: ¡Aparta el auto!

-
Input sentence: Run.
Decoded sentence: ¡Aparece a la cama!

-
Input sentence: Who?
Decoded sentence: ¿Quién está ahí?

-
Input sentence: Fire!
Decoded sentence: ¡Aparece!

-
Input sentence: Fire!
Decoded sentence: ¡Aparece!

-
Input sentence: Fire!
Decoded sentence: ¡Aparece!

-
Input sentence: Help!
Decoded sentence: ¡Aparta el auto!

-
Input sentence: Help!
Decoded sentence: ¡Aparta el auto!

-
Input sentence: Help!
Decoded sentence: ¡Aparta el auto!

-
Input sentence: Jump!
Decoded sentence: ¡Qué comida!

-
Input sentence: Jump.
Decoded sentence: ¡Vallame a Tomás!

-
Input sentence: Stop!
Decoded sentence: ¡Deja de comprar!

-
Input sentence: Stop!
Decoded sentence: ¡Deja de comprar!



-
Input sentence: I tried.
Decoded sentence: Yo conocí.

-
Input sentence: I'll go.
Decoded sentence: Voy a estar aquí.

-
Input sentence: I'm Tom.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm fat.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm fat.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm fit.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm hit!
Decoded sentence: Estoy cansado.

-
Input sentence: I'm old.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm shy.
Decoded sentence: Estoy cansado.

-
Input sentence: I'm wet.
Decoded sentence: Soy un carro.

-
Input sentence: It's OK.
Decoded sentence: Es muy difícil.

-
Input sentence: It's me!
Decoded sentence: Es muy bueno.

-
Input sentence: It's me.
Decoded sentence: Es muy difícil.

-
Input sentence: Join us.
Decoded sentence: Alguien está bien.

-
Input sentence: Join us.
Decoded sentence: Alguien está bien.

-
Input sentence: Keep it.
Decoded sentence: Sigue aquí.

-
Input sentence: Me

-
Input sentence: I agreed.
Decoded sentence: Yo te desperté.

-
Input sentence: I am fat.
Decoded sentence: Estoy cansado.

-
Input sentence: I am old.
Decoded sentence: Estoy cansado.

-
Input sentence: I ate it.
Decoded sentence: Yo estoy cansado.

-
Input sentence: I ate it.
Decoded sentence: Yo estoy cansado.

-
Input sentence: I can go.
Decoded sentence: Puedo ayudar.

-
Input sentence: I did OK.
Decoded sentence: Yo tengo aquellos.

-
Input sentence: I did it.
Decoded sentence: Yo tengo aquellos.

-
Input sentence: I failed.
Decoded sentence: Yo te desperté.

-
Input sentence: I forgot.
Decoded sentence: Lo siento.

-
Input sentence: I get by.
Decoded sentence: Me encanta el trabajo.

-
Input sentence: I get it.
Decoded sentence: Yo conozco.

-
Input sentence: I got it.
Decoded sentence: Me encantan las noticias.

-
Input sentence: I phoned.
Decoded sentence: Yo puedo contar.

-
Input sentence: I refuse.
Decoded sentence: Yo conocía.

-
Input sentence: I resign.
Decoded sentence

-
Input sentence: Watch me.
Decoded sentence: Los precios están de acuerdo.

-
Input sentence: Watch me.
Decoded sentence: Los precios están de acuerdo.

-
Input sentence: Watch me.
Decoded sentence: Los precios están de acuerdo.

-
Input sentence: Watch us.
Decoded sentence: Los precios están de acuerdo.

-
Input sentence: Watch us.
Decoded sentence: Los precios están de acuerdo.

-
Input sentence: We agree.
Decoded sentence: Nosotros lo sabíamos.

-
Input sentence: We tried.
Decoded sentence: Vamos a casa.

-
Input sentence: We'll go.
Decoded sentence: Vamos a casa.

-
Input sentence: We're OK.
Decoded sentence: Estamos de acuerdo.

-
Input sentence: What for?
Decoded sentence: ¿Qué estás haciendo?

-
Input sentence: What fun!
Decoded sentence: ¡Qué tarde!

-
Input sentence: Who am I?
Decoded sentence: ¿Quién está ahí?

-
Input sentence: Who came?
Decoded sentence: ¿Quién estaba escondiendo?

-
Input sentence: Who died?
Decoded sentence: ¿Quién lo ha decidido?

-
Input sentence: Who 

-
Input sentence: I hate it.
Decoded sentence: Yo tengo un montón.

-
Input sentence: I hate it.
Decoded sentence: Yo tengo un montón.

-
Input sentence: I hate it.
Decoded sentence: Yo tengo un montón.

-
Input sentence: I hit Tom.
Decoded sentence: Voy a estar aquí.

-
Input sentence: I hope so.
Decoded sentence: Espero que te vaya.

-
Input sentence: I knew it.
Decoded sentence: Sabía que esto.

-
Input sentence: I laughed.
Decoded sentence: Yo lo hace dos.

-
Input sentence: I like it.
Decoded sentence: Me gusta el trabajo.

-
Input sentence: I love it!
Decoded sentence: Me encanta el trabajo.

-
Input sentence: I love it.
Decoded sentence: Me encanta el trabajo.

-
Input sentence: I mean it!
Decoded sentence: Yo tengo algo.

-
Input sentence: I mean it.
Decoded sentence: Yo tengo algo.

-
Input sentence: I mean it.
Decoded sentence: Yo tengo algo.

-
Input sentence: I miss it.
Decoded sentence: Yo te de la comida.

-
Input sentence: I miss it.
Decoded sentence: Yo te de la comida.

-
Input sentence: Leave Tom.
Decoded sentence: Deja de comer.

-
Input sentence: Leave Tom.
Decoded sentence: Deja de comer.

-
Input sentence: Leave Tom.
Decoded sentence: Deja de comer.

-
Input sentence: Leave now.
Decoded sentence: Deja de comer.

-
Input sentence: Leave now.
Decoded sentence: Deja de comer.

-
Input sentence: Leave now.
Decoded sentence: Deja de comer.

-
Input sentence: Let me go!
Decoded sentence: ¡Vete a la cama!

-
Input sentence: Let me go!
Decoded sentence: ¡Vete a la cama!

-
Input sentence: Let me go!
Decoded sentence: ¡Vete a la cama!

-
Input sentence: Let me go.
Decoded sentence: Déjame ir.

-
Input sentence: Let me in.
Decoded sentence: Déjame ir.

-
Input sentence: Let me in.
Decoded sentence: Déjame ir.

-
Input sentence: Let us in.
Decoded sentence: ¡Vete a casa!

-
Input sentence: Let's eat.
Decoded sentence: Vamos a la cama.

-
Input sentence: Let's see.
Decoded sentence: Vamos a casa.

-
Input sentence: Let's try!
Decoded sentence: ¡Vamos a casa!

-
Input sentence: Tom cares.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom cares.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom cried.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom dozed.
Decoded sentence: Tom no está aquí.

-
Input sentence: Tom drove.
Decoded sentence: Tom no está aquí.

-
Input sentence: Tom is OK.
Decoded sentence: Tom es muy listo.

-
Input sentence: Tom is in.
Decoded sentence: Tom es un carro.

-
Input sentence: Tom is up.
Decoded sentence: Tom es un carro.

-
Input sentence: Tom knits.
Decoded sentence: Tom sabe.

-
Input sentence: Tom knows.
Decoded sentence: Tom no está aquí.

-
Input sentence: Tom moved.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom moved.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom rocks.
Decoded sentence: Tom se está poniendo.

-
Input sentence: Tom stood.
Decoded sentence: Tom se ve a sus diez.

-
Input sentence: Tom swims.
Decoded sentence: Tom está aq

-
Input sentence: Cut it out!
Decoded sentence: ¡Ayúdame a Tomás!

-
Input sentence: Did Tom go?
Decoded sentence: ¿Viste?

-
Input sentence: Do come in!
Decoded sentence: ¡Sal de mi casa!

-
Input sentence: Do come in!
Decoded sentence: ¡Sal de mi casa!

-
Input sentence: Do come in!
Decoded sentence: ¡Sal de mi casa!

-
Input sentence: Do come in.
Decoded sentence: Solo estoy de verdad.

-
Input sentence: Do come in.
Decoded sentence: Solo estoy de verdad.

-
Input sentence: Do come in.
Decoded sentence: Solo estoy de verdad.

-
Input sentence: Do come in.
Decoded sentence: Solo estoy de verdad.

-
Input sentence: Do men cry?
Decoded sentence: ¿Sabe Tom?

-
Input sentence: Don't come.
Decoded sentence: No te estás escondiendo.

-
Input sentence: Don't jump!
Decoded sentence: ¡No te puedes ir?

-
Input sentence: Don't look.
Decoded sentence: ¡No te puedes ir?

-
Input sentence: Don't move!
Decoded sentence: ¡No te puedes ir?

-
Input sentence: Don't move.
Decoded sentence: ¡No te pued